Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = '/home/smart/Documents/Deepankar/ML/tensorflow_tutorial/notMNIST/notMNIST_large/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [34]:
batch_size = 128
hidden_nodes = 1024
reg = 1e-2

def nn_feedforward(x, w1, w2, b1, b2):
    z1 = tf.matmul(x, w1) + b1
    a1 = tf.nn.relu(z1)
    z2 = tf.matmul(a1, w2) + b2
    
    return z2

new_graph = tf.Graph()
with new_graph.as_default():
    
    # declare placeholders for training data and define constants 
    # for validation and test data
    x_train = tf.placeholder(dtype=tf.float32, shape=(batch_size, image_size*image_size))
    y_train = tf.placeholder(dtype=tf.float32, shape=(batch_size, num_labels))
    x_val = tf.constant(valid_dataset)
    x_test = tf.constant(test_dataset)
    
    # declare variables
    w1 = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_nodes]))
    b1 = tf.Variable(tf.zeros([hidden_nodes]))
    w2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    # feedforward and activations and cross-entropy loss
    z2 = nn_feedforward(x_train, w1, w2, b1, b2)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_train, logits=z2) +
                         reg*tf.nn.l2_loss(w1) +
                         reg*tf.nn.l2_loss(w2))
    
    # initialise the SGD optimiser
    optimiser = tf.train.GradientDescentOptimizer(0.3).minimize(loss)
    
    # predictions on each dataset
    y_train_pred = tf.nn.softmax(z2)
    y_val_pred = tf.nn.softmax(nn_feedforward(x_val, w1, w2, b1, b2))
    y_test_pred = tf.nn.softmax(nn_feedforward(x_test, w1, w2, b1, b2))
    

In [35]:
num_steps = 5001

with tf.Session(graph=new_graph) as new_session:

    # randomly initialise global variables
    tf.global_variables_initializer().run()
    print("Initialised")
    
    for step in xrange(num_steps):
        # generate mini-batch
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # initialise the feed_dict
        feed_dict = {x_train: batch_data, y_train: batch_labels}
        
        # run the session
        _, l, y_pred = new_session.run([optimiser, loss, y_train_pred], feed_dict=feed_dict)
        
        # print the predictions after every 500 steps
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(y_pred, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(y_val_pred.eval(), valid_labels))
            print("----####----")
        
    print("Test accuracy: %.1f%%" % accuracy(y_test_pred.eval(), test_labels))
    new_session.close()
    

Initialised
Minibatch loss at step 0: 3604.448242
Minibatch accuracy: 10.2%
Validation accuracy: 33.9%
----####----
Minibatch loss at step 500: 154.109192
Minibatch accuracy: 84.4%
Validation accuracy: 83.3%
----####----
Minibatch loss at step 1000: 8.188718
Minibatch accuracy: 85.2%
Validation accuracy: 84.4%
----####----
Minibatch loss at step 1500: 1.059434
Minibatch accuracy: 86.7%
Validation accuracy: 84.5%
----####----
Minibatch loss at step 2000: 0.706521
Minibatch accuracy: 85.2%
Validation accuracy: 83.9%
----####----
Minibatch loss at step 2500: 0.765250
Minibatch accuracy: 82.8%
Validation accuracy: 84.0%
----####----
Minibatch loss at step 3000: 0.754346
Minibatch accuracy: 83.6%
Validation accuracy: 84.4%
----####----
Minibatch loss at step 3500: 0.738489
Minibatch accuracy: 78.1%
Validation accuracy: 84.1%
----####----
Minibatch loss at step 4000: 0.679435
Minibatch accuracy: 83.6%
Validation accuracy: 84.0%
----####----
Minibatch loss at step 4500: 0.887424
Minibatch acc

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

The loss(accuracy) on the training set decreases(increases) however, the validation set accuracy and the test accuracy are very poor.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [32]:
batch_size = 64
hidden_nodes = 1024
# reg = 1e-2

def nn_feedforward(x, w1, w2, b1, b2, istrain=True):
    if(istrain is True):
        x = tf.nn.dropout(x, keep_prob=0.5)
    z1 = tf.matmul(x, w1) + b1
    a1 = tf.nn.relu(z1)
    if(istrain is True):
        a1 = tf.nn.dropout(a1, keep_prob=0.5)
    z2 = tf.matmul(a1, w2) + b2
    
    return z2

new_graph = tf.Graph()
with new_graph.as_default():
    
    # declare placeholders for training data and define constants 
    # for validation and test data
    x_train = tf.placeholder(dtype=tf.float32, shape=(batch_size, image_size*image_size))
    y_train = tf.placeholder(dtype=tf.float32, shape=(batch_size, num_labels))
    x_val = tf.constant(valid_dataset)
    x_test = tf.constant(test_dataset)
    
    # declare variables
    w1 = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_nodes]))
    b1 = tf.Variable(tf.zeros([hidden_nodes]))
    w2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    # feedforward and activations and cross-entropy loss
    z2 = nn_feedforward(x_train, w1, w2, b1, b2)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_train, logits=z2))
    
    # initialise the SGD optimiser
    optimiser = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    
    # predictions on each dataset
    y_train_pred = tf.nn.softmax(z2)
    y_val_pred = tf.nn.softmax(nn_feedforward(x_val, w1, w2, b1, b2, istrain=False))
    y_test_pred = tf.nn.softmax(nn_feedforward(x_test, w1, w2, b1, b2, istrain=False))
    

In [33]:
num_steps = 5001

with tf.Session(graph=new_graph) as new_session:

    # randomly initialise global variables
    tf.global_variables_initializer().run()
    print("Initialised")
    
    for step in xrange(num_steps):
        # generate mini-batch
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # initialise the feed_dict
        feed_dict = {x_train: batch_data, y_train: batch_labels}
        
        # run the session
        _, l, y_pred = new_session.run([optimiser, loss, y_train_pred], feed_dict=feed_dict)
        
        # print the predictions after every 500 steps
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(y_pred, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(y_val_pred.eval(), valid_labels))
            print("----####----")
        
    print("Test accuracy: %.1f%%" % accuracy(y_test_pred.eval(), test_labels))
    new_session.close()
    

Initialised
Minibatch loss at step 0: 890.854065
Minibatch accuracy: 7.8%
Validation accuracy: 16.7%
----####----
Minibatch loss at step 500: 115.175316
Minibatch accuracy: 65.6%
Validation accuracy: 79.4%
----####----
Minibatch loss at step 1000: 120.467133
Minibatch accuracy: 60.9%
Validation accuracy: 81.0%
----####----
Minibatch loss at step 1500: 57.628067
Minibatch accuracy: 68.8%
Validation accuracy: 79.8%
----####----
Minibatch loss at step 2000: 42.809052
Minibatch accuracy: 73.4%
Validation accuracy: 79.1%
----####----
Minibatch loss at step 2500: 79.214966
Minibatch accuracy: 62.5%
Validation accuracy: 80.4%
----####----
Minibatch loss at step 3000: 27.216545
Minibatch accuracy: 76.6%
Validation accuracy: 80.2%
----####----
Minibatch loss at step 3500: 48.289368
Minibatch accuracy: 62.5%
Validation accuracy: 79.9%
----####----
Minibatch loss at step 4000: 40.074249
Minibatch accuracy: 67.2%
Validation accuracy: 79.6%
----####----
Minibatch loss at step 4500: 49.335449
Miniba

#### Inference: 

Adding dropout mitigates overfitting since it behaves like regularisation. Adding both dropout and L2-reg shows almost no improvement in performance over just L2-reg.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [37]:
from IPython.core.debugger import Tracer

batch_size = 128
input_dim = image_size*image_size
hidden_layers = 2
hidden_nodes = 1024
params = {}
reg = 1e-2
l2_pen = 0

def nn_feedforward(x, w, b):
    z = tf.matmul(x, w) + b
    return z
    
def nn_relu(z):
    a = tf.nn.relu(z)
    return a

# computes forward pass
def nn_computeforward(a):
    for h in xrange(hidden_layers):
        z = nn_feedforward(a, params['w%s' % (h+1)], params['b%s' %(h+1)])
        a = nn_relu(z)
    score = tf.matmul(a, params['w%s' % (hidden_layers+1)]) + params['b%s' % (hidden_layers+1)]
    
    return score

new_graph = tf.Graph()
with new_graph.as_default():
    
    # declare placeholders for training data and define constants 
    # for validation and test data
    x_train = tf.placeholder(dtype=tf.float32, shape=(batch_size, input_dim))
    y_train = tf.placeholder(dtype=tf.float32, shape=(batch_size, num_labels))
    x_val = tf.constant(valid_dataset)
    x_test = tf.constant(test_dataset)
    
    # declare variables
    input_dim = image_size*image_size
    temp = np.repeat(hidden_nodes, hidden_layers)
    dims = [input_dim] + temp.tolist() + [num_labels]
    # Tracer()()
    for h in xrange(hidden_layers+1):
        params['w%s' % (h+1)] = tf.Variable(tf.truncated_normal([dims[h], dims[h+1]]))
        params['b%s' %(h+1)] = tf.Variable(tf.zeros([dims[h+1]]))
    
    # compute forward pass
    score = nn_computeforward(x_train)
    
    # compute L2-penalty and cross-entropy loss
    for h in xrange(hidden_layers):
        l2_pen += tf.nn.l2_loss(params['w%s' % (h+1)])
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_train, logits=score) + reg*l2_pen)
    
    # initialise the SGD optimiser
    optimiser = tf.train.AdagradOptimizer(0.5,initial_accumulator_value=1e-2).minimize(loss)
    
    # predictions on each dataset
    y_train_pred = tf.nn.softmax(score)
    y_val_pred = tf.nn.softmax(nn_computeforward(x_val))
    y_test_pred = tf.nn.softmax(nn_computeforward(x_test))
    

In [ ]:
num_steps = 20001

with tf.Session(graph=new_graph) as new_session:

    # randomly initialise global variables
    tf.global_variables_initializer().run()
    print("Initialised")
    
    for step in xrange(num_steps):
        # generate mini-batch
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # initialise the feed_dict
        feed_dict = {x_train: batch_data, y_train: batch_labels}
        
        # run the session
        _, l, y_pred = new_session.run([optimiser, loss, y_train_pred], feed_dict=feed_dict)
        
        # print the predictions after every 500 steps
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(y_pred, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(y_val_pred.eval(), valid_labels))
            print("----####----")
        
    print("Test accuracy: %.1f%%" % accuracy(y_test_pred.eval(), test_labels))
    new_session.close()
    